In [1]:
import random

In [2]:
def toss(p):
  return random.randint(0, 100) <= p

In [3]:
class SwapOperator:
  def __init__(self, i, j):
    self.i = i
    self.j = j
  
  def get_max(self):
    return max(self.i, self.j)

  def __str__(self):
    pair = (self.i, self.j)
    return str(pair)

  def __repr__(self):
    pair = (self.i, self.j)
    return str(pair)

In [4]:
class SwapSequence:
  def __init__(self, seq):
    self.seq = seq # List of SwapOperator objects

  def iterate(self):
    for so in self.seq:
      yield so
  
  def add_swap_operator(self, so):
    self.seq.append(so)

  def get_highest_index(self):
    hi = 0
    for so in self.seq:
      hi = max(hi, so.get_max())
    return hi

  def copy(self):
    ret = []
    for so in self.seq:
      ret.append(SwapOperator(so.i, so.j))
    return SwapSequence(ret)

  def __add__(self, b): # Returns *Basic SwapSequence* that is equivalent to list(Sequence a) + list(Sequence b)
    c_seq = self.seq + b.seq
    c = SwapSequence(c_seq)
    n = c.get_highest_index() + 1

    initial_solution = Solution.gen_random_Solution(n)
    resultant_solution = initial_solution.add(c)

    return resultant_solution.sub(initial_solution)

  def __str__(self):
    return str(self.seq)

In [5]:
class Solution:
  def __init__(self, perm):
    self.perm = perm # A basic permutation

  def __add_SO(self, so, p):
    ret = self.copy()
    if toss(p):
      ret.perm[so.i], ret.perm[so.j] = ret.perm[so.j], ret.perm[so.i]
    return ret

  def __add_SS(self, ss, p):
    ret = self.copy()
    for so in ss.iterate():
      ret = ret.add(so, p)
    return ret

  def add(self, obj, p = 100):
    if isinstance(obj, SwapOperator):
      return self.__add_SO(obj, p)
    elif isinstance(obj, SwapSequence):
      return self.__add_SS(obj, p)
    else: return None
  
  def sub(self, b, p = 100):
    b = b.copy()
    a = self.copy()
    n = len(a.perm)
    ss = []
    for i in range(n):
      for j in range(n):
        if a.perm[i] == b.perm[j]:
          if i != j:
            so = SwapOperator(i, j)
            ss.append(so)
            b = b.add(so, p)
          else:
            break
    return SwapSequence(ss)

  def get(self, ind):
    return self.perm[ind]

  def copy(self):
    return Solution(self.perm[:])

  def __str__(self):
    return str(self.perm)

  def __eq__(self, b):
    for i in range(len(self.perm)):
      if self.get(i) != b.get(i):
        return False
    return True

  @classmethod
  def gen_random_Solution(cls, n):
    perm = [i for i in range(n)]
    random.shuffle(perm)
    return Solution(perm)

In [6]:
class Particle:
  def __init__(self, position, velocity, tsp):
    self.position = position
    self.p_best = self.position.copy()
    self.velocity = velocity
    self.fitness = tsp.apply_solution(position)
    self.p_best_fitness = self.fitness

  def get_fitness(self):
    return self.fitness
  
  def get_position(self):
    return self.position

  def get_p_best(self):
    return self.p_best

  def get_p_best_fitness(self):
    return self.p_best_fitness
  
  def update_velocity(self, g_best, alpha, beta):
    ini = self.velocity.copy()
    term1 = self.p_best.sub(self.position, alpha)
    term2 = g_best.sub(self.position, beta)
    self.velocity += term1 + term2

  def update_position(self, tsp):
    initial = self.position
    new = self.position.add(self.velocity)

    n_fitness = tsp.apply_solution(new)

    if n_fitness < self.get_p_best_fitness():
      self.p_best = new.copy()
      self.p_best_fitness = n_fitness

    self.fitness = n_fitness
    self.position = new

  def __str__(self):
    ret = f"Particle: {self.position}, fitness: {self.get_fitness()}, best_fitness: {self.get_p_best_fitness()}"
    return ret
  
  def __repr__(self):
    ret = f"Particle: {self.position}, fitness: {self.get_fitness()}, best_fitness: {self.get_p_best_fitness()}"
    return ret

  @classmethod
  def gen_random_Particle(self, n, tsp):
    position = Solution.gen_random_Solution(n)
    velocity = Solution.gen_random_Solution(n).sub(Solution.gen_random_Solution(n))
    return Particle(position, velocity, tsp)

In [7]:
class TSP:
  def __init__(self, g):
    self.g = g
    self.n = len(g)

  def apply_solution(self, s):
    cost = 0
    for i in range(self.n):
      cost += self.g[s.get(i)][s.get((i + 1) % self.n)]
    return cost
  
  @classmethod
  def gen_random_TSP(cls, n, soln):
    ret = []
    for i in range(n):
      row = []
      for j in range(n):
        row.append(random.randint(2, 10))
      ret.append(row)
      ret[i][i] = 0
    soln = soln.perm
    for i in range(n):
      ret[soln[i]][soln[(i+1)%n]] = 1
    return TSP(ret)

In [8]:
def get_g_best(particles, g_best):
  g_best, g_best_fitness = g_best
  for particle in particles:
    if particle.get_p_best_fitness() < g_best_fitness:
      g_best = particle.get_p_best().copy()
      g_best_fitness = particle.get_p_best_fitness()
  return g_best, g_best_fitness

In [9]:
ALPHA = 70
BETA = 80
POPULATION_SIZE = 20
NUM_NODES = 10
NUM_ITERATIONS = 1000

In [10]:
actual_solution = [i for i in range(NUM_NODES)]
random.shuffle(actual_solution)
actual_solution = Solution(actual_solution)

In [11]:
tsp = TSP.gen_random_TSP(NUM_NODES, actual_solution)
particles = [Particle.gen_random_Particle(NUM_NODES, tsp) for _ in range(POPULATION_SIZE)]
g_best_fitness = particles[0].get_fitness()
g_best, g_best_fitness = get_g_best(particles, (particles[0], g_best_fitness))

In [12]:
print("Initial Particles: ")
for particle in particles:
  print(particle)
print(f"Best Solution: {g_best}, Fitness: {g_best_fitness}")

Initial Particles: 
Particle: [0, 2, 3, 7, 8, 9, 1, 5, 4, 6], fitness: 67, best_fitness: 67
Particle: [3, 9, 4, 7, 6, 5, 8, 1, 0, 2], fitness: 50, best_fitness: 50
Particle: [1, 0, 4, 9, 7, 6, 2, 8, 3, 5], fitness: 71, best_fitness: 71
Particle: [1, 8, 2, 3, 7, 6, 4, 0, 5, 9], fitness: 67, best_fitness: 67
Particle: [6, 2, 3, 8, 5, 9, 0, 4, 7, 1], fitness: 51, best_fitness: 51
Particle: [4, 8, 7, 3, 5, 9, 1, 0, 2, 6], fitness: 63, best_fitness: 63
Particle: [4, 7, 2, 9, 6, 8, 1, 0, 3, 5], fitness: 62, best_fitness: 62
Particle: [0, 5, 9, 7, 3, 6, 8, 4, 1, 2], fitness: 47, best_fitness: 47
Particle: [2, 6, 7, 4, 1, 8, 5, 3, 0, 9], fitness: 54, best_fitness: 54
Particle: [1, 5, 6, 2, 4, 8, 7, 0, 3, 9], fitness: 66, best_fitness: 66
Particle: [5, 7, 1, 6, 4, 2, 3, 9, 8, 0], fitness: 47, best_fitness: 47
Particle: [3, 8, 4, 1, 0, 5, 7, 9, 6, 2], fitness: 57, best_fitness: 57
Particle: [6, 5, 1, 3, 0, 4, 2, 8, 7, 9], fitness: 59, best_fitness: 59
Particle: [3, 5, 2, 7, 1, 6, 9, 0, 8, 4], fi

In [13]:
for i in range(NUM_ITERATIONS):
  for particle in particles:
    particle.update_velocity(g_best, ALPHA, BETA)
    particle.update_position(tsp)
    g_best, g_best_fitness = get_g_best(particles, (g_best, g_best_fitness))
  print(f"Iteration {i}, Best Solution: {g_best}, Fitness: {g_best_fitness}")

Iteration 0, Best Solution: [1, 2, 9, 0, 6, 8, 4, 5, 7, 3], Fitness: 43
Iteration 1, Best Solution: [1, 9, 5, 3, 7, 2, 6, 8, 0, 4], Fitness: 39
Iteration 2, Best Solution: [1, 9, 5, 3, 7, 2, 6, 8, 0, 4], Fitness: 39
Iteration 3, Best Solution: [1, 9, 5, 3, 7, 2, 6, 8, 0, 4], Fitness: 39
Iteration 4, Best Solution: [1, 4, 7, 0, 2, 9, 5, 3, 6, 8], Fitness: 27
Iteration 5, Best Solution: [1, 4, 7, 0, 2, 9, 5, 3, 6, 8], Fitness: 27
Iteration 6, Best Solution: [1, 4, 7, 0, 2, 9, 5, 3, 6, 8], Fitness: 27
Iteration 7, Best Solution: [1, 4, 7, 0, 2, 9, 5, 3, 6, 8], Fitness: 27
Iteration 8, Best Solution: [1, 4, 7, 0, 2, 9, 5, 3, 6, 8], Fitness: 27
Iteration 9, Best Solution: [1, 4, 7, 0, 2, 9, 5, 3, 6, 8], Fitness: 27
Iteration 10, Best Solution: [1, 4, 7, 0, 2, 9, 5, 3, 6, 8], Fitness: 27
Iteration 11, Best Solution: [1, 4, 7, 0, 2, 9, 5, 3, 6, 8], Fitness: 27
Iteration 12, Best Solution: [1, 4, 7, 0, 2, 9, 5, 3, 6, 8], Fitness: 27
Iteration 13, Best Solution: [1, 4, 7, 0, 2, 9, 5, 3, 6, 8], 

In [14]:
print(actual_solution)
print(g_best)

[1, 2, 9, 5, 3, 6, 7, 0, 4, 8]
[1, 6, 7, 0, 2, 9, 5, 3, 4, 8]
